In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id_train_features = '1LnKdehYxXYXIUhxO1NFeu5GebmKpbYjY'
id_test_features = '1CIuprbEUeJDEdg0WO5Sq2-xV8OBElXLX'
id_train_labels = '1dzgH430IP8VV4Jw6InbfTiMEFUxZk1pO'
id_test_labels = '10dwdJ5EI_w-HfeM3F8CyctlqAnuW3RY9'

In [ ]:
downloaded = drive.CreateFile({'id':id_train_features})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_features.csv')  
# Dataset is now stored in a Pandas Dataframe
train_X = pd.read_csv('train_features.csv',header=None)
train_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0.087,0.106,0.059,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072
1,0.106,0.059,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228
2,0.059,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228,0.123
3,0.170,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228,0.123,0.019
4,0.084,0.090,0.052,0.153,0.159,0.049,0.086,0.055,0.080,0.052,0.098,0.038,0.059,0.177,0.047,0.046,0.205,0.090,0.127,0.181,0.171,0.106,0.193,0.059,0.082,0.205,0.079,0.066,0.101,0.061,0.126,0.165,0.075,0.165,0.199,0.056,0.072,0.228,0.123,0.019,0.123


In [ ]:
downloaded = drive.CreateFile({'id':id_test_features})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_features.csv')  
# Dataset is now stored in a Pandas Dataframe
test_X = pd.read_csv('test_features.csv',header=None)
test_X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000,0.000,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.000,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.000,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182,0.096
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.090,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182,0.096,0.236
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.09,0.077,0.077,0.073,0.067,0.054,0.092,0.085,0.083,0.182,0.098,0.229,0.096,0.113,0.148,0.050,0.045,0.069,0.133,0.131,0.154,0.182,0.096,0.236,0.082


In [ ]:
downloaded = drive.CreateFile({'id':id_train_labels})   # replace the id with id of file you want to access
downloaded.GetContentFile('train_labels.csv')  
# Dataset is now stored in a Pandas Dataframe
train_Y = pd.read_csv('train_labels.csv',header=None)
train_Y.head()

,0
0,1
1,1
2,1
3,1
4,1


In [ ]:
downloaded = drive.CreateFile({'id':id_test_labels})   # replace the id with id of file you want to access
downloaded.GetContentFile('test_labels.csv')  
# Dataset is now stored in a Pandas Dataframe
test_Y = pd.read_csv('test_labels.csv',header=None)
test_Y.head()

,0,1,2,3,4,5
0,c,l,a,s,s,2
1,c,l,a,s,s,2
2,c,l,a,s,s,2
3,c,l,a,s,s,2
4,c,l,a,s,s,2


In [ ]:
from sklearn import svm
from sklearn.utils.validation import column_or_1d
from sklearn.metrics import accuracy_score
x = np.array(train_X)
y = np.array(train_Y)
y = column_or_1d(y, warn=True)
x_t = np.array(test_X)
y_t = np.array(test_Y)
y_t = column_or_1d(y_t, warn=True)
print('Shape of training dataset',x.shape)
print('Shape of label tensor:', y.shape)
print('Shape of test dataset',x_t.shape)
print('Shape of test label tensor:', y_t.shape)

Shape of training dataset (10452, 41)
Shape of label tensor: (10452,)
Shape of test dataset (258829, 41)
Shape of test label tensor: (258829,)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [ ]:
clf = svm.SVC(probability=True)
clf.fit(x, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [ ]:
results = clf.predict(x_t)
preds_train = clf.predict_proba(x)
preds_test = clf.predict_proba(x_t) 

In [ ]:
accuracy_score(y_t, results)

0.6269506121802425

In [ ]:
a = np.zeros(preds_test.shape[0])
j = 0
for i in preds_test:
  a[j] = i[1]
  j = j+1
a.shape

(258829,)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_t, a)

0.6669937044647569